<a href="https://colab.research.google.com/github/AnnaZhuravleva/avtobreja/blob/master/udpipe_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с UDPipe и графовый метод поиска тональных слов

## UDPipe parsing train dataset

In [0]:
!pip install ufal.udpipe

     |████████████████████████████████| 307kB 2.8MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp36-cp36m-linux_x86_64.whl size=5625757 sha256=06fc3dce8f980bb7101f68fb36961bbd707df4c3f5ac760106a9abe1668668fa
  Stored in directory: /root/.cache/pip/wheels/0c/9d/db/6d3404c33da5b7adb6c6972853efb6a27649d3ba15f7e9bebb
Successfully built ufal.udpipe


In [0]:
!wget https://github.com/AnnaZhuravleva/avtobreja/raw/master/SentiRuEval_rest_train/SentiRuEval_rest_train.csv


--2019-12-20 21:28:24--  https://github.com/AnnaZhuravleva/avtobreja/raw/master/SentiRuEval_rest_train/SentiRuEval_rest_train.csv
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/SentiRuEval_rest_train/SentiRuEval_rest_train.csv [following]
--2019-12-20 21:28:24--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/SentiRuEval_rest_train/SentiRuEval_rest_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36571759 (35M) [text/plain]
Saving to: ‘SentiRuEval_rest_train.csv’

SentiRuEval_rest_tr 100%[===================>]  34.88M  52.8MB/s    in 0.

In [0]:
!wget -O "russian-syntagrus-ud-2.4-190531.udpipe" https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2998/russian-syntagrus-ud-2.4-190531.udpipe?sequence=74&isAllowed=y

--2019-12-21 15:46:16--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2998/russian-syntagrus-ud-2.4-190531.udpipe?sequence=74
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45859472 (44M) [application/octet-stream]
Saving to: ‘russian-syntagrus-ud-2.4-190531.udpipe’

russian-syntagrus-u 100%[===================>]  43.73M  15.9MB/s    in 2.8s    

2019-12-21 15:46:21 (15.9 MB/s) - ‘russian-syntagrus-ud-2.4-190531.udpipe’ saved [45859472/45859472]



In [0]:
import sys
import ufal.udpipe

In [0]:
ud_model = ufal.udpipe.Model.load("russian-syntagrus-ud-2.4-190531.udpipe")

In [0]:
pipe = ufal.udpipe.Pipeline(ud_model, "tokenize", "tag", "parse", "conllu")

In [0]:
# Usage example

a = "Положа руку на сердце, очень хочется спать. А не проект по автобрее пилить!"
print(pipe.process(a))

# newdoc
# newpar
# sent_id = 1
# text = Положа руку на сердце, очень хочется спать.
1	Положа	Положать	VERB	_	Aspect=Perf|Tense=Past|VerbForm=Conv|Voice=Act	7	advcl	_	_
2	руку	рука	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	1	obj	_	_
3	на	на	ADP	_	_	4	case	_	_
4	сердце	сердце	NOUN	_	Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing	1	obl	_	SpaceAfter=No
5	,	,	PUNCT	_	_	1	punct	_	_
6	очень	очень	ADV	_	Degree=Pos	7	advmod	_	_
7	хочется	хотеться	VERB	_	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Mid	0	root	_	_
8	спать	спать	VERB	_	Aspect=Imp|VerbForm=Inf|Voice=Act	7	xcomp	_	SpaceAfter=No
9	.	.	PUNCT	_	_	7	punct	_	_

# sent_id = 2
# text = А не проект по автобрее пилить!
1	А	а	CCONJ	_	_	3	cc	_	_
2	не	не	PART	_	_	3	advmod	_	_
3	проект	проект	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	0	root	_	_
4	по	по	ADP	_	_	5	case	_	_
5	автобрее	автобрея	NOUN	_	Animacy=Inan|Case=Dat|Gender=Fem|Number=Sing	6	obl	_	_
6	пилить	пилить	VERB	_	Aspect=Perf|VerbForm=Inf|

In [0]:
import pandas as pd

In [0]:
with open("SentiRuEval_rest_train.csv", 'r') as f:
    train = pd.read_csv(f)

#А то у нас уже записано id
train.drop(["Unnamed: 0"], axis=1, inplace=True)


In [0]:
train.head()

,food,service,text
0,8,8,И пускай на меня не обижается наш прославленны...
1,9,10,"- Здравствуйте. Виа Д’Арженто! - Добрый вечер,..."
2,9,1,"Советую вам уволить Вашего метродотеля Елену, ..."
3,8,9,отличный средне вековый интеръер. Приятное обс...
4,10,8,Ужинали в ресторане Баден-Баден 6 марта . Импо...


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import re
from os import path, mkdir

re.sub(r'(foo)', r'\g<1>123', 'foobar')

d = "/content/drive/My Drive/parsed_train"
if not path.exists(d):
    mkdir(d)

for i, text in enumerate(train.text):
    # Avoiding lemmas like "5рублей"
    pattern = '([0-9])(\w)'
    text = re.sub(pattern, r'\g<1> \g<2>', text)

    # Getting rid of russian punctuation and *
    text = re.sub('»|«', '"', text)
    text = re.sub('\*', '', text)

    parsed_text = open(path.join(d, "{}.conllu".format(i)), 'w', encoding='utf-8')
    parsed = pipe.process(text)
    parsed_text.write(parsed)


Распарсенные тексты [здесь](https://drive.google.com/open?id=181szxLRYHorRrzGTcA1XwwA23MVy7F90)

## Расширение списка сидов при помощи графа

Попробуем реализовать подход из статьи Automatic Generation of Domain-specific Sentiment Russian Dictionaries (Dubatovka et al. 2016).

Изменение: мы хотим собирать союзные пары не регулярками по паттерну, а при помощи синтаксиеской разметки UDPipe.

In [0]:

import ufal.udpipe
# ufal.udpipe.Model etc. are SWIG-magic and cannot be detected by pylint
# pylint: disable=no-member

class Model:
    def __init__(self, path):
        """Load given model."""
        self.model = ufal.udpipe.Model.load(path)
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % path)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, in_format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % in_format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, out_format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()

        return output

In [0]:
model = Model("russian-syntagrus-ud-2.4-190531.udpipe")

In [0]:
def get_pos(word):
    a = model.read(word, 'horizontal')
    model.tag(a[0])
    pos = a[0].words[1].upostag
    return pos

In [0]:
!wget https://github.com/AnnaZhuravleva/avtobreja/raw/master/seeds/Food_words.txt

--2019-12-21 15:46:30--  https://github.com/AnnaZhuravleva/avtobreja/raw/master/seeds/Food_words.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/seeds/Food_words.txt [following]
--2019-12-21 15:46:30--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/seeds/Food_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503 [text/plain]
Saving to: ‘Food_words.txt’

Food_words.txt      100%[===================>]     503  --.-KB/s    in 0s      

2019-12-21 15:46:31 (91.1 MB/s) - ‘Food_words.txt’ saved [503/503]



In [0]:
with open('Food_words.txt', 'r') as f:
    food_keys = f.read()
food_keys

'Food\tвкусный\t1\nFood\tбольшой\t1\nFood\tпрекрасный\t1\nFood\tразнообразный\t1\nFood\tединственный\t1\nFood\tдостойный\t1\nFood\tстранный\t1\nFood\tотличный\t1\nFood\tгорячий\t1\nFood\tсытный\t1\nFood\tсвежий\t1\nFood\tвеликолепный\t1\nFood\tинтересный\t1\nFood\tразличный\t1\nFood\tнеобычный\t1\nFood\tприятный\t1\nFood\tплохой\t0\nFood\tневкусный\t0\nFood\tсочно\t1\nFood\tпонравиться\t1'

In [0]:
food = {}
for line in food_keys.split('\n'):
    print(line)
    aspect, lemma, value = line.split('\t')
    pos = get_pos(lemma)
    food[(lemma, pos)] = value

Food	вкусный	1
Food	большой	1
Food	прекрасный	1
Food	разнообразный	1
Food	единственный	1
Food	достойный	1
Food	странный	1
Food	отличный	1
Food	горячий	1
Food	сытный	1
Food	свежий	1
Food	великолепный	1
Food	интересный	1
Food	различный	1
Food	необычный	1
Food	приятный	1
Food	плохой	0
Food	невкусный	0
Food	сочно	1
Food	понравиться	1


In [0]:
food

{('большой', 'ADJ'): '1',
 ('великолепный', 'ADJ'): '1',
 ('вкусный', 'ADJ'): '1',
 ('горячий', 'ADJ'): '1',
 ('достойный', 'ADJ'): '1',
 ('единственный', 'ADJ'): '1',
 ('интересный', 'ADJ'): '1',
 ('невкусный', 'ADJ'): '0',
 ('необычный', 'ADJ'): '1',
 ('отличный', 'ADJ'): '1',
 ('плохой', 'ADJ'): '0',
 ('понравиться', 'VERB'): '1',
 ('прекрасный', 'ADJ'): '1',
 ('приятный', 'ADJ'): '1',
 ('различный', 'ADJ'): '1',
 ('разнообразный', 'ADJ'): '1',
 ('свежий', 'ADJ'): '1',
 ('сочно', 'ADV'): '1',
 ('странный', 'ADJ'): '1',
 ('сытный', 'ADJ'): '1'}

In [0]:
# POS, которые мы будем включать в граф
targets = set([i[1] for i in food])
targets

{'ADJ', 'ADV', 'VERB'}

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_d = "/content/drive/My Drive/parsed_train"

In [0]:
import os

example_path = os.path.join(train_d, "1.conllu")
with open(example_path, 'r') as f:
    ex = f.read()
print(ex)

# newdoc
# newpar
# sent_id = 1
# text = - Здравствуйте.
1	-	-	PUNCT	_	_	2	punct	_	_
2	Здравствуйте	здравствовать	VERB	_	Aspect=Imp|Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voice=Act	0	root	_	SpaceAfter=No
3	.	.	PUNCT	_	_	2	punct	_	_

# sent_id = 2
# text = Виа Д’Арженто!
1	Виа	Виа	NOUN	_	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	2	nsubj	_	_
2	Д’Арженто	Д’Арженто	ADJ	_	Degree=Pos|Gender=Neut|Number=Sing|Variant=Short	0	root	_	SpaceAfter=No
3	!	!	PUNCT	_	_	2	punct	_	_

# sent_id = 3
# text = - Добрый вечер, я хотела бы заказать у вас столик на 2 0.0 0 1 4 февраля.
1	-	-	PUNCT	_	_	3	punct	_	_
2	Добрый	добрый	ADJ	_	Case=Nom|Degree=Pos|Gender=Masc|Number=Sing	3	amod	_	_
3	вечер	вечер	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	0	root	_	SpaceAfter=No
4	,	,	PUNCT	_	_	6	punct	_	_
5	я	я	PRON	_	Case=Nom|Number=Sing|Person=1	6	nsubj	_	_
6	хотела	хотеть	VERB	_	Aspect=Imp|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	3	acl:relcl	_	_
7	бы	бы	AUX	_	_	6	aux	_	_
8	за

In [0]:
from nltk.parse import DependencyGraph

In [0]:
example = "Нам всё понравилось, еда была горячая и вкусная, яблоки нам принесли сочные, не сладкие и спелые, официанты только бегали, ходили мимо, но не приносили счет"
parsed_ex = [sent for sent in pipe.process(example).split('\n') if sent and not sent.startswith('#')]
parsed_ex

['1\tНам\tмы\tPRON\t_\tCase=Dat|Number=Plur|Person=1\t3\tiobj\t_\t_',
 '2\tвсё\tвсе\tPRON\t_\tAnimacy=Inan|Case=Nom|Gender=Neut|Number=Sing\t3\tnsubj\t_\t_',
 '3\tпонравилось\tпонравиться\tVERB\t_\tAspect=Perf|Gender=Neut|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Mid\t0\troot\t_\tSpaceAfter=No',
 '4\t,\t,\tPUNCT\t_\t_\t7\tpunct\t_\t_',
 '5\tеда\tеда\tNOUN\t_\tAnimacy=Inan|Case=Nom|Gender=Fem|Number=Sing\t7\tnsubj\t_\t_',
 '6\tбыла\tбыть\tAUX\t_\tAspect=Imp|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act\t7\tcop\t_\t_',
 '7\tгорячая\tгорячий\tADJ\t_\tCase=Nom|Degree=Pos|Gender=Fem|Number=Sing\t3\tobl\t_\t_',
 '8\tи\tи\tCCONJ\t_\t_\t9\tcc\t_\t_',
 '9\tвкусная\tвкусный\tADJ\t_\tCase=Nom|Degree=Pos|Gender=Fem|Number=Sing\t7\tconj\t_\tSpaceAfter=No',
 '10\t,\t,\tPUNCT\t_\t_\t13\tpunct\t_\t_',
 '11\tяблоки\tяблоко\tNOUN\t_\tAnimacy=Inan|Case=Nom|Gender=Neut|Number=Plur\t13\tnsubj\t_\t_',
 '12\tнам\tмы\tPRON\t_\tCase=Dat|Number=Plur|Person=1\t13\tiobj\t_\t_',
 '13\t

In [0]:
g = DependencyGraph(parsed_ex, top_relation_label="root")

In [0]:
print(pipe.process("Вася встал, но не лег спать обратно"))

# newdoc
# newpar
# sent_id = 1
# text = Вася встал, но не лег спать обратно
1	Вася	Вася	PROPN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	2	nsubj	_	_
2	встал	встать	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	SpaceAfter=No
3	,	,	PUNCT	_	_	6	punct	_	_
4	но	но	CCONJ	_	_	6	cc	_	_
5	не	не	PART	_	_	6	advmod	_	_
6	лег	лечь	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Mid	2	conj	_	_
7	спать	спать	VERB	_	Aspect=Imp|VerbForm=Inf|Voice=Act	6	xcomp	_	_
8	обратно	обратно	ADV	_	Degree=Pos	2	advmod	_	SpacesAfter=\n




In [0]:
class ConnectionGraph:
    self.node_map = {}
    self.edges = DefaultDict(int)

    def __init__(targets, cc, K=3):
        NEG = ('не', 'PART') 
        self.targets = targets
        self.
        for tree in trees:
            tree = 


In [0]:
from collections import defaultdict
nodes = defaultdict(list)
edges = defaultdict({int})

for i in g.triples():
    print(i)



TypeError: ignored

In [0]:
deps = g.root['deps']
-

{'address': 14,
 'ctag': 'NOUN',
 'deps': defaultdict(list, {'acl': [2]}),
 'feats': 'Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur',
 'head': 0,
 'lemma': 'яблоко',
 'rel': 'root',
 'tag': '_',
 'word': 'яблоки'}